# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 29 2022 9:34AM,243050,10,8326059-BO,Eywa Pharma Inc.,Released
1,Jul 29 2022 9:34AM,243050,10,8326063-BO,Eywa Pharma Inc.,Released
2,Jul 29 2022 9:34AM,243050,10,8334910,Eywa Pharma Inc.,Released
3,Jul 29 2022 9:34AM,243050,10,8330515-BO,Eywa Pharma Inc.,Released
4,Jul 29 2022 9:34AM,243050,10,8336039,Eywa Pharma Inc.,Released
5,Jul 29 2022 9:34AM,243050,10,8336040,Eywa Pharma Inc.,Released
6,Jul 29 2022 9:34AM,243050,10,8336041,Eywa Pharma Inc.,Released
7,Jul 29 2022 9:34AM,243050,10,8336042,Eywa Pharma Inc.,Released
8,Jul 29 2022 9:32AM,243051,10,SOTAH0000740,"Nextsource Biotechnology, LLC",Released
9,Jul 29 2022 9:32AM,243051,10,SOTAH0000741,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,243047,Released,10
30,243048,Released,1
31,243049,Released,8
32,243050,Released,8
33,243051,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243047,NaN,NaN,10.0
243048,NaN,NaN,1.0
243049,NaN,NaN,8.0
243050,NaN,NaN,8.0
243051,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242964,17.0,11.0,6.0
242965,0.0,0.0,1.0
242967,0.0,1.0,0.0
242976,0.0,0.0,1.0
243003,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242964,17,11,6
242965,0,0,1
242967,0,1,0
242976,0,0,1
243003,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242964,17,11,6
1,242965,0,0,1
2,242967,0,1,0
3,242976,0,0,1
4,243003,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242964,17,11,6
1,242965,,,1
2,242967,,1,
3,242976,,,1
4,243003,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.
8,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC"
10,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC"
18,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC"
19,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc."
29,Jul 29 2022 9:06AM,243046,16,Sartorius Bioprocess Solutions
31,Jul 29 2022 9:01AM,243045,102,"Senseonics, Incorporated"
32,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation
47,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation
54,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,,,8
1,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",,,2
2,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",,,8
3,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",,,1
4,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc.",,,10
5,Jul 29 2022 9:06AM,243046,16,Sartorius Bioprocess Solutions,,,2
6,Jul 29 2022 9:01AM,243045,102,"Senseonics, Incorporated",,,1
7,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,,,15
8,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,,4,3
9,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,8,,
1,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",2,,
2,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",8,,
3,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",1,,
4,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc.",10,,
5,Jul 29 2022 9:06AM,243046,16,Sartorius Bioprocess Solutions,2,,
6,Jul 29 2022 9:01AM,243045,102,"Senseonics, Incorporated",1,,
7,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,15,,
8,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,3,4,
9,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,8,,
1,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",2,,
2,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",8,,
3,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",1,,
4,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc.",10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,8.0,NaN,NaN
1,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
2,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",8.0,NaN,NaN
3,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",1.0,NaN,NaN
4,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc.",10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 9:34AM,243050,10,Eywa Pharma Inc.,8.0,0.0,0.0
1,Jul 29 2022 9:32AM,243051,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
2,Jul 29 2022 9:31AM,243049,15,"Virtus Pharmaceuticals, LLC",8.0,0.0,0.0
3,Jul 29 2022 9:28AM,243048,15,"Mizner Bioscience, LLC",1.0,0.0,0.0
4,Jul 29 2022 9:27AM,243047,15,"Alliance Pharma, Inc.",10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2187358,120.0,4.0,0.0
102,243045,1.0,0.0,0.0
12,243024,0.0,1.0,0.0
15,729144,19.0,0.0,0.0
16,971981,4.0,1.0,0.0
20,972052,28.0,14.0,17.0
21,729019,3.0,0.0,0.0
22,1215087,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2187358,120.0,4.0,0.0
1,102,243045,1.0,0.0,0.0
2,12,243024,0.0,1.0,0.0
3,15,729144,19.0,0.0,0.0
4,16,971981,4.0,1.0,0.0
5,20,972052,28.0,14.0,17.0
6,21,729019,3.0,0.0,0.0
7,22,1215087,3.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,120.0,4.0,0.0
1,102,1.0,0.0,0.0
2,12,0.0,1.0,0.0
3,15,19.0,0.0,0.0
4,16,4.0,1.0,0.0
5,20,28.0,14.0,17.0
6,21,3.0,0.0,0.0
7,22,3.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,120.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,19.0
4,16,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0
Executing,4.0,0.0,1.0,0.0,1.0,14.0,0.0,2.0
Released,120.0,1.0,0.0,19.0,4.0,28.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,4.0,0.0,1.0,0.0,1.0,14.0,0.0,2.0
2,Released,120.0,1.0,0.0,19.0,4.0,28.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,4.0,0.0,1.0,0.0,1.0,14.0,0.0,2.0
2,Released,120.0,1.0,0.0,19.0,4.0,28.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()